<a href="https://colab.research.google.com/github/RobinHamers/mombasa_solutions/blob/main/create_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai import types
from google.genai.types import (
    HttpOptions,
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)
import os
import pandas as pd
import geopandas as gpd
import sys
from pydantic import BaseModel
from typing import List

In [85]:
OUTPUT_RISK_EXPLAIN_JSON = '{"fire": ["string"], "explanation_fire": ["string"], "heat": ["string"], "explanation_heat": ["string"], "flood": ["string"], "explanation_flood": ["string"]}'
class RiskActions_and_explanation(BaseModel):
    fire:List[str]
    explanation_fire:List[str]
    heat:List[str]
    explanation_heat:List[str]
    flood:List[str]
    explanation_flood:List[str]

In [86]:
OUTPUT_RISK_EXPLAIN = """
    Output the solutions and explain (chain of thoughts) why you propose this solution refering to the inputs you considered.
    Format: 2 times 1 section: 🌡 Heat. Max 3 actions each. Only if risk is relevant.
    Each solution = 1 bullet, 1–2 lines. explanation for that solution in its seperate section also following the bullets
    """

In [87]:
 system_instruction = f"""
    You are an environmental expert creating actionable mitigation recommendations for climate and disaster resilience in urban settings.

    Below is a description of the current risks and environmental conditions for a specific local zone. You are tasked with generating mitigation actions tailored to that zone's risks, demographic context, and relevant points of interest.

    Use clear language and address actions that can be taken both by individuals and by local authorities. Use bullet points where helpful. Be specific, not generic.
    When referring to community members, use 'community members at-risk' instead of terms like 'elderly' or 'vulnerable' that may not be appropriate in all contexts.

    🎯 TASK:
    Based on this information, list **concise practical and location-specific mitigation actions** that can reduce climate and disaster risks in the local area. Structure them by risk type (e.g., Heat). Include targeted suggestions related to nearby POIs or vulnerable populations.

    {OUTPUT_RISK_EXPLAIN}
    """

In [88]:
def prepare_prompt_risk_action(heat_risk, lst_day, lst_night,
                           canopy_cover_pct, comments
                          ):
    prompt = f"""

            INPUT

            🟨 LOCAL CONTEXT (Buffer zone of ~500m radius):
            - Heat Risk (monthly): {heat_risk}
            - LST Day/Night: {lst_day}/{lst_night}
            - Canopy Cover = {canopy_cover_pct}%


        """
    return prompt

In [89]:
import geopandas as gpd

lst_day_df = gpd.read_file("/content/LST-August-2024-Day.geojson")
lst_night_df = gpd.read_file("/content//LST-August-2024-Night-.geojson")
canopy_df = gpd.read_file("/content//Mombasa-Canopy-Cover-.geojson")
heat_df = gpd.read_file("/content//Heat-Retention-Risk-202408-.geojson")
# Displaying the first few rows to check if it loaded correctly
df.head()

,felt:feature,h3,metric,date,label,geometry
0,1,623665692029255679,0.0,2024-08-01,heat_retention_risk_max,POINT Z (39.73854 -4.14051 0)
1,2,623665692029288447,0.0,2024-08-01,heat_retention_risk_max,POINT Z (39.73737 -4.14006 0)
2,3,623665692029321215,0.0,2024-08-01,heat_retention_risk_max,POINT Z (39.73949 -4.13973 0)
3,4,623665692029353983,0.0,2024-08-01,heat_retention_risk_max,POINT Z (39.73832 -4.13928 0)
4,5,623665692029386751,0.0,2024-08-01,heat_retention_risk_max,POINT Z (39.73877 -4.14174 0)


In [90]:
heat_risk = heat_df['metric'].mean()
lst_day = lst_day_df['metric'].mean()
lst_night = lst_night_df['metric'].mean()
canopy_cover_pct = canopy_df['metric'].mean()


In [91]:
PROMPT_TEMPLATE = prepare_prompt_risk_action(
            heat_risk=heat_risk,
            lst_day=lst_day,
            lst_night=lst_night,
            canopy_cover_pct=canopy_cover_pct,
            comments="" # Add an empty string for comments to avoid NameError
        )